In [2]:
import librosa
import numpy as np
import os
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Definice cest k tónům
tone_folders = {
    "C4": "data/c4",
    "D4": "data/d4",
    "E4": "data/e4",
    "F4": "data/f4",
    "G4": "data/g4",
    "A4": "data/a4",
    "B4": "data/b4",
    "C5": "data/c5",
    "D5": "data/d5",
    "E5": "data/e5",
    "F5": "data/f5",
    "G5": "data/g5",
    "A5": "data/a5",
    "B5": "data/b5",
    "C6": "data/c6",
    "D6": "data/d6",
    "E6": "data/e6"
}

# 2. Funkce pro extrakci features
def extract_features(file_path, n_mfcc=20, sr=22050):
    try:
        y, sr = librosa.load(file_path, sr=sr, duration=2.5)  # Normalizace délky
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc_mean = np.mean(mfcc.T, axis=0)
        
        # Přidání dalších features
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
        
        return np.concatenate([mfcc_mean, [spectral_centroid, zero_crossing_rate, chroma]])
    except Exception as e:
        print(f"Chyba při zpracování {file_path}: {str(e)}")
        return None

# 3. Načtení dat
def load_data():
    features = []
    for tone, folder in tone_folders.items():
        for file in os.listdir(folder):
            if file.endswith((".wav", ".mp3")):
                file_path = os.path.join(folder, file)
                feat = extract_features(file_path)
                if feat is not None:
                    features.append(feat)
    return np.array(features)

# 4. Příprava modelu
def train_model(X):
    # Normalizace
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Rozdělení na train/test (80/20)
    X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)
    
    # Trénink modelu
    model = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
    model.fit(X_train)
    
    # Testování (simulace - potřebuješ i negativní data)
    test_scores = model.decision_function(X_test)
    return model, scaler, test_scores

# 5. Spuštění
if __name__ == "__main__":
    print("Načítám data...")
    X = load_data()
    print(f"Načteno {len(X)} vzorků")
    
    print("Trénuji model...")
    model, scaler, _ = train_model(X)
    
    # Ukázka použití
    test_file = "temp.wav"
    features = extract_features(test_file)
    if features is not None:
        scaled = scaler.transform([features])
        prediction = model.predict(scaled)
        print("🔔 Je to kalimba!" if prediction[0] == 1 else "🔕 Není to kalimba")


from joblib import dump

# Po tréninku modelu (v části 5)
dump({
    'model': model,
    'scaler': scaler
}, 'kalimba_filter.joblib')

Načítám data...


C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=309
  warnings.warn(
C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1956
  warnings.warn(


Načteno 1106 vzorků
Trénuji model...
🔕 Není to kalimba


['kalimba_filter.joblib']

In [7]:
import librosa
import numpy as np
import os
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from joblib import dump

# Nastavení cest
tone_folders = {
    "C4": "data/c4",
    "D4": "data/d4",
    # ... všechny ostatní tóny
}

def extract_features(file_path, n_mfcc=20, sr=22050, duration=2.5):
    y, sr = librosa.load(file_path, sr=sr, duration=duration)
    if len(y) < sr * 0.5:  # Minimálně 0.5s
        return None

    try:
        chroma = librosa.feature.chroma_stft(
            y=y, 
            sr=sr,
            n_fft=2048,
            hop_length=512,
            tuning=0.0
        )

    except Exception as e:
        print(f"Chyba u souboru {file_path}: {str(e)}")
        return None
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc.T, axis=0)
    
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    
    return np.concatenate([mfcc_mean, [spectral_centroid, zero_crossing_rate, chroma]])

def train_kalimba_filter():
    features = []
    for folder in tone_folders.values():
        for file in os.listdir(folder):
            if file.endswith((".wav", ".mp3")):
                feat = extract_features(os.path.join(folder, file))
                if feat is not None:
                    features.append(feat)
    
    X = np.array(features)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
    model.fit(X_scaled)
    
    dump({'model': model, 'scaler': scaler}, 'kalimba_filter.joblib')
    print("Model uložen jako 'kalimba_filter.joblib'")

if __name__ == "__main__":
    train_kalimba_filter()


Model uložen jako 'kalimba_filter.joblib'


NameError: name 'X_scaled' is not defined

In [6]:
import matplotlib.pyplot as plt

class KalimbaDetector:
    def __init__(self, model_path='kalimba_filter.joblib'):
        self.svm_data = load(model_path)
        self.ocsvm = self.svm_data['model']
        self.scaler = self.svm_data['scaler']
        self.rms_threshold = 0.005  # Experimentujte s touto hodnotou
        
    def is_kalimba(self, buffer, debug=False):
        if len(buffer) < 512:
            return False
        
        # Kontrola energie
        rms = librosa.feature.rms(y=buffer).mean()
        if rms < self.rms_threshold:
            if debug: print("Příliš nízká energie")
            return False
            
        # Extrakce features
        features = self._extract_features(buffer)
        if features is None:
            return False
            
        # Transformace
        try:
            scaled = self.scaler.transform([features])
            score = self.ocsvm.decision_function(scaled)[0]
            
            if debug:
                print(f"RMS: {rms:.4f}")
                print(f"SVM skóre: {score:.2f}")
                self._plot_features(features)
                
            return score > self._find_optimal_threshold()
        except:
            return False
    
    def _extract_features(self, buffer):
        try:
            mfcc = librosa.feature.mfcc(y=buffer, sr=22050, n_mfcc=20)
            if mfcc.shape[1] < 5:  # Minimální počet rámců
                return None
                
            mfcc_mean = np.mean(mfcc.T, axis=0)
            
            spectral_centroid = librosa.feature.spectral_centroid(y=buffer, sr=22050)[0].mean()
            zero_crossing = librosa.feature.zero_crossing_rate(buffer)[0].mean()
            chroma = librosa.feature.chroma_stft(y=buffer, sr=22050, n_fft=2048, tuning=0).mean()
            
            return np.concatenate([mfcc_mean, [spectral_centroid, zero_crossing, chroma]])
        except:
            return None
    
    def _find_optimal_threshold(self):
        """Dynamický threshold na základě trénovacích dat"""
        return np.percentile(self.ocsvm.decision_function(self.scaler.transform(X)), 5)  # 5% percentil
    
    def _plot_features(self, features):
        plt.figure(figsize=(10,4))
        plt.plot(features, marker='o')
        plt.title("Normalizované hodnoty features")
        plt.xlabel("Index feature")
        plt.ylabel("Hodnota")
        plt.show()

In [8]:
# GPT

In [34]:
import os
import numpy as np
import librosa
import librosa.display
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import joblib

# Cesty ke složkám jednotlivých tónů kalimby
tone_folders = {
    "C4": "data/c4", "D4": "data/d4", "E4": "data/e4", "F4": "data/f4",
    "G4": "data/g4", "A4": "data/a4", "B4": "data/b5", "C5": "data/c5",
    "D5": "data/d5", "E5": "data/e5", "F5": "data/f5", "G5": "data/g5",
    "A5": "data/a5", "B5": "data/b5", "C6": "data/c6", "D6": "data/d6", "E6": "data/e6"
}

# Funkce na extrakci MFCC příznaků z audio souboru
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)
    return np.mean(mfccs, axis=1)  # Vezmeme průměr přes časové osy

# Načtení datasetu
features = []
for tone, folder in tone_folders.items():
    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_path = os.path.join(folder, file)
            feature = extract_features(file_path)
            features.append(feature)

# Převést na numpy pole
X_train = np.array(features)

# Normalizace dat
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Trénink One-Class SVM
model = OneClassSVM(kernel="rbf", gamma="auto", nu=0.5)  # Parametr nu určuje citlivost
model.fit(X_train_scaled)

# Uložení modelu
joblib.dump(model, "kalimba_ocsvm.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Trénink dokončen, model uložen jako 'kalimba_ocsvm.pkl'")


C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=672
  warnings.warn(


Trénink dokončen, model uložen jako 'kalimba_ocsvm.pkl'


In [36]:
# Načtení modelu
model = joblib.load("kalimba_ocsvm.pkl")
scaler = joblib.load("scaler.pkl")

# Testovací zvuk
test_file = "temp_segment.wav"
test_feature = extract_features(test_file)
test_feature_scaled = scaler.transform([test_feature])

# Predikce (-1 = jiný zvuk, 1 = tón kalimby)
prediction = model.predict(test_feature_scaled)
if prediction[0] == 1:
    print("Zvuk odpovídá tónu kalimby!")
else:
    print("Toto není tón kalimby.")


Toto není tón kalimby.
